## Code from last lecture

In [4]:
import gzip
import itertools

In [24]:
!head ./data/GCA_000001405.15_GRCh38_genomic.fna.gz

��`�^ GCA_000001405.15_GRCh38_genomic.fna ��ˮ$˖e���+�"�
}v���3�sqA�MCQw[8�]���ښͻ�6:��2_M?���,G�3�i�i �K�'Țkk:Y1M���Z�	I�:	S]p�u��z[�� Ɵ�������m�C6B���p-{�̨UXM#��t5ԇ��.YҧwW	�}c"ɛ�)��q3���LȎL���JW2�l����x�v������Nƈ��/���3��c�'��~j)]��s��>�2�Jdzؖ� ��c���"Iy劾��o�\�X��e�kh�s�^����ϱ"��� �$�c�Z�Z�����h�\����q�n�=Z��+�*~���K���(K��j�]�i�,Kv?}�z�M�<xIŨ�p�u3;�m��	B��w��5�Բ��^��j�kno�nb�����$�L��_7w��9�!l%~41�^�F�LpP�WA;��Ƨa���I~�|�w�+����4�Xl`5f��T���ɩ�2���X��qG�-��g�	�sbL����v
��eP+��
��:n����h$}&s�k�\`2�˳<�x��$/�ʥ�vH��/���H��g��ʗɏޗ&T勦��ų)���������b"%�$mn�xD�䏬T�u�42�i�iւ7��ohϚ6N?*�(�l�"
WH�\]�������Y���ඌo-��%�r%Wo��ƞ��D�/��V.��f6���"{n�H�]�(F�؇)odh+���P���z�������;H��z�QKH�/�f�MN�|��Ľ�18�C'���l�,i�ogPr]��H�ܜ�o�����1$"���쳘Ӎ�����Mv��aZ�Vk�F���6����-��+h�����
͌��X��0+�;����!�&��~m�OʢB΀��_=N]�`�g��7���~D�X7`���� ms�
���g�u36�i�8-� >����gI�`@Z̠�Kr l�%�ꁺ
���i%�n�V�	ڙ�_

In [25]:
def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into a list of headers 
        and fragment sequences for each sequence contained"""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'r')
    else:
        fp = open(filename, 'r')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

In [26]:
header, seq = loadFasta("./data/GCA_000001405.15_GRCh38_genomic.fna.gz")
print(len(header), "sequences")
for i in range(len(header)):
    if header[i].startswith("CM") or header[i].startswith("J0"):
        print(header[i])
        print(len(seq[i])-1, "bases", seq[i][:30], "...", seq[i][-30:])

TypeError: a bytes-like object is required, not 'str'

## Output each chromosome as a simple string to speed up future processing

In [ ]:
header, seq = loadFasta("data/GCA_000001405.15_GRCh38_genomic.fna.gz")
print(len(header), "sequences")
for i in range(len(header)):
    if header[i].startswith("CM") or header[i].startswith("J0"):
        start = header[i].find('chromosome ')
        chromo = header[i][start+11:header[i].find(',')] if (start >= 0) else "MT"
        with open("data/Chr%s.seq" % chromo, 'w') as fp:
            fp.write(seq[i])            

## A function to construct the reverse-complement version of a DNA sequence

In [7]:
def revComp(dnaSeq):
    return ''.join([{'A':'T','C':'G','G':'C','T':'A'}[base] for base in reversed(dnaSeq)])

In [8]:
print(revComp("GAGACAT"))
print(revComp("ATGTCTC"))

ATGTCTC
GAGACAT


In [10]:
test = "cGTGATATTCCTTTTAAGAGGGAAGGgaaagaacagaaccaaaccaaaccaaaccacagataaggtaaaagaaaggttgatgaggtggagtccttcgacc".upper()
print(revComp(test))

GGTCGAAGGACTCCACCTCATCAACCTTTCTTTTACCTTATCTGTGGTTTGGTTTGGTTTGGTTCTGTTCTTTCCCTTCCCTCTTAAAAGGAATATCACG


## Find the frequency of all k-mers in the genome

# Warning: This next cell takes a while...

In [ ]:
import time

DATA = "../HumanGenome/"  # change to match the directory where your chromosome sequences are.
chromo = [str(i) for i in xrange(1,23)] + ['X', 'Y', 'MT']

kmerCount = {}
K = 11
L = 0
for contig in chromo:
    tick = time.time()
    with open(DATA+"Chr%s.seq" % contig, 'r') as fp:
        seq = fp.read()
    for i in xrange(1,len(seq)-K+1):
        kmer = seq[i:i+K]
        for base in "RYSWKMBDHVN":
            if (base in kmer):
                break
        else:
            kmerCount[kmer] = kmerCount.get(kmer,0) + 1
            kmer = revComp(kmer)
            kmerCount[kmer] = kmerCount.get(kmer,0) + 1
    tock = time.time()
    print(contig, len(seq)-1, len(kmerCount), "%6.2f secs" % (tock - tick))
    tick = tock
    L += len(seq) - 1
    break
print(L, len(kmerCount))

In [ ]:
print(4**K - len(kmerCount))

## The code belows prints out all of the missing k-mers, if you're interested

In [20]:
missing = []
for pattern in itertools.product("ACGT", repeat=K):
    kmer = ''.join(pattern)
    if kmer not in kmerCount:
        missing.append(kmer)
print(len(missing))

167156


## Plot the kmer distribution

In [ ]:
import matplotlib
import matplotlib.pyplot as plot
%matplotlib inline

# Compute a histogram of kmer-counts (i.e. how many kmers appear 1 time, 2 times, 3 times ...)
maxcount = 500
hist = [0 for i in range(maxcount)]
for kmer in kmerCount:
    count = kmerCount[kmer]
    if (count < maxcount):
        hist[count] += 1

fig = plot.figure(figsize=(10,6))
plot.plot([i for i in range(maxcount)], hist)
plot.show()